# Solar prediction: Convolutional models

#### **TODO**:

- [ ] Normalize w.r.t. last time-slice
- [ ] Use clearsky model
- [ ] Use a more flexible architecture (no weights, or at least not constraint to sum up to one)
- [ ] Weights derived from the convolutional layer, not directly from the input only using dense.
- [ ] Add some MaxPooling AveragePooling in the architecture.

In [5]:
import pandas as pd
import numpy as np
import feather
import pandas_profiling
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

df = (feather.read_dataframe('/home/SHARED/SOLAR/data/oahu_min.feather')
             .set_index('Datetime'))

In [6]:
# We load the info of the sensors to extract the longitude information
info = pd.read_csv('/home/SHARED/SOLAR/data/info.csv')

info.Location = info.Location.apply(lambda x: (x[:2] + x[-2:]).replace('_', ''))
info.index = info.Location
# Sorted longitudes
longs = info['Longitude'].sort_values(ascending=False)

# We drop two sensors (they are different compared to the other 17, since they are "tilted")
df.drop('GT_AP6', inplace=True, axis=1)
df.drop('GT_DH1', inplace=True, axis=1)

# Just some auxiliar code to homogeneize name of sensors across different tables
homogen_name = lambda x: x[-4:].replace('_', '')
df.columns = [homogen_name(x) for x in df.columns.values.tolist()]

# Finally, we sort the data according to sensor's longitude
df = df[longs.index]

In [116]:
pandas_profiling.ProfileReport(df,bins=20,correlation_threshold=0.95)

Number of variables,18
Number of observations,532837
Total Missing (%),0.0%
Total size in memory,73.2 MiB
Average record size in memory,144.0 B
Numeric,9
Categorical,0
Boolean,0
Date,1
Text (Unique),0
Rejected,8


## Stacking the time slices in the dataframe

In [7]:
# https://stackoverflow.com/questions/15722324/sliding-window-in-numpy
def window_stack_forward(a, stepsize=1, width=3):
    return np.hstack( a[i:1+i-width or None:stepsize] for i in range(0, width) )

# I feel this function can also be done for pd.DataFrame
def window_stack(a, width=3):
    n = a.shape[0]
    return np.hstack(list(a[(width-1-i):(n-i)] for i in range(0, width)))

In [8]:
# In pandas 0.24, use df.to_numpy() instead of df.values. Also care with non-numeric columns
width = 10
a = window_stack(df.to_numpy(), width=width)

In [9]:
times   = [ ('t' if not idx else 't-{:d}'.format(idx)) for idx in range(width) ]
columns = pd.MultiIndex.from_product((times, df.columns), names=('time', 'location'))

In [10]:
# Convert back to DataFrame, just for convenience of having indexes
df_roll = pd.DataFrame(a, index=df.index[width-1:], columns=columns)

In [11]:
# Split target (time t) and variables (times t-1 to t-width+1)
y = df_roll['t']
X = df_roll.drop(columns='t', level='time')

In [12]:
# Split train-test, approximately 12 and 4 months respectively
X_train, X_test = X[:'2011-07-31'], X['2011-08-01':]
y_train, y_test = y[:'2011-07-31'], y['2011-08-01':]

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(449936, 153)
(82892, 153)
(449936, 17)
(82892, 17)


In this case, we are going to select more than one time slice as inputs. In this case X is a numpy array of rank 3.

X : (item,sensor,time_slice)

In [14]:
longs_np = longs.index.to_numpy()
n_sensors = len(longs_np)
print('Number of sensors considered:',n_sensors)

chosen_times = times[1:10:1]
print('Chosen times slices as features:',chosen_times)
print('Number of time slices considered:',len(chosen_times))

Number of sensors considered: 17
Chosen times slices as features: ['t-1', 't-2', 't-3', 't-4', 't-5', 't-6', 't-7', 't-8', 't-9']
Number of time slices considered: 9


# Training Models

In [1]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [2]:
import keras

Using TensorFlow backend.


In [3]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape, Add, Multiply, Subtract, Dropout
from keras.layers import Conv2D, MaxPooling2D, LocallyConnected1D, Conv1D, UpSampling1D, MaxPooling1D, Dot, Concatenate
from keras.layers import LocallyConnected2D, Conv2D
from keras import backend as K
from clr import CyclicLR

In [4]:
from sklearn.model_selection import TimeSeriesSplit

In [15]:
lr = 0.0001
opt = keras.optimizers.Adam(lr=lr)

# We add a callback to log metrics and another one to schedule the learning rate

#see clr.py in this same folder
from clr import CyclicLR

c1 = keras.callbacks.BaseLogger(stateful_metrics=None)
c2 = CyclicLR(step_size=250, base_lr=lr)
c3 = keras.callbacks.History()
c4 = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)

batch_size = 1 << 11   # as big as possible so we can explore many models
epochs = 1 << 5

In [17]:
def train_and_test_sensor(X_train, y_train,id_sensor,chosen_times, to_array , make_model_sensor, longs_np= longs_np,verbose = 0):
    # Preprocessing
    X_tr, y_tr = to_array(X_train,y_train,chosen_times, sensor=longs_np[id_sensor], sort_time = True)
    time_slices = len(chosen_times)
    n_sensors = len(longs_np)
    
    # Validation using TS split (just to obtain different MAE estimations, no hyperoptimization for the moment)
    mae_val_list = []
    for tr_idx, va_idx in TimeSeriesSplit(n_splits=5).split(X_tr):
        model = make_model_sensor(id_sensor, n_sensors=n_sensors,time_slices=time_slices)
        model.compile(opt, loss='mean_absolute_error')
        
        model.fit(np.atleast_3d(X_tr[tr_idx]), y_tr[tr_idx], batch_size=batch_size, epochs=epochs, validation_data=
              (np.atleast_3d(X_tr[va_idx]),y_tr[va_idx]), callbacks=[c2, c3,c4], verbose=verbose)
        mae_val_list.append(c3.history['val_loss'][-1])
        if verbose==1:
            print('MAE_val ', c3.history['val_loss'][-1])
    
#     # Testing
#     model = make_model_sensor_conv1d_time(id_sensor, n_sensors=n_sensors,time_slices=len(chosen_times))
#     model.compile(opt, loss='mean_absolute_error')
#     model.fit(np.atleast_3d(X_tr), y_tr, batch_size=batch_size, epochs=epochs, validation_data=
#               (np.atleast_3d(X_te),y_te), callbacks=[c2, c3,c4], verbose=0)
    
#     print('MAE_test ', c3.history['val_loss'][-1])
    return longs_np[id_sensor], np.array(mae_val_list)

In [18]:
def print_mae_sensors(mae_val_list,longs_np):
    """
    mae_val_list : (k_folds_val, n_sensors)
    longs_np : (n_sensors)
    """
    print('====MAE VALIDATION=====')
    print('* MEAN: {:0.2f} (+/- {:0.2f})'.format(mae_val_list.mean(), mae_val_list.std()),
      '/ List: [',', '.join(['{:6.2f}'.format(mae) for mae in mae_val_list.mean(axis=1)]),']')
    print('====DISAGGREGATED======')
    for sensor,mae_list in zip(longs_np,mae_val_list.T):
        print("* {:4}: {:0.2f} (+/- {:0.2f})".format(sensor, mae_list.mean(), mae_list.std()),'/ List: [',', '.join(['{:6.2f}'.format(mae) for mae in mae_list]),']') 

## Convolutional 1D with last time slice

First we preprocess the dataset (for the moment, we'll just use as features the t-1 values at each sensor)

In [15]:
# We only use the previous timestep as features
X_tr1 = X_train['t-1']
y_tr1 = y_train

X_te1 = X_test['t-1']
y_te1 = y_test

Now, in order to use a 1D convolution, we are going to sort the sensors. For the initial test, we'll just sort them by longitude (from East to West). That way, nearer sensors are in close positions in the tensor, so the 1D convolution may extract useful correlations.

Note: many other possible ordenations of the sensors could be added as new channels in the input tensor

In [16]:
# We load the info of the sensors to extract the longitude information
info = pd.read_csv('/home/SHARED/SOLAR/data/info.csv')

info.Location = info.Location.apply(lambda x: (x[:2] + x[-2:]).replace('_', ''))
info.index = info.Location
# Sorted longitudes
longs = info['Longitude'].sort_values(ascending=False)

# We drop two sensors (they are different compared to the other 17, since they are "tilted")
X_tr1.drop('GT_AP6', inplace=True, axis=1)
y_tr1.drop('GT_AP6', inplace=True, axis=1)
X_tr1.drop('GT_DH1', inplace=True, axis=1)
y_tr1.drop('GT_DH1', inplace=True, axis=1)
X_te1.drop('GT_AP6', inplace=True, axis=1)
y_te1.drop('GT_AP6', inplace=True, axis=1)
X_te1.drop('GT_DH1', inplace=True, axis=1)
y_te1.drop('GT_DH1', inplace=True, axis=1)

# Just some auxiliar code to homogeneize name of sensors across different tables
homogen_name = lambda x: x[-4:].replace('_', '')
X_tr1.columns = [homogen_name(x) for x in X_tr1.columns.values.tolist()]
y_tr1.columns = [homogen_name(x) for x in y_tr1.columns.values.tolist()]
X_te1.columns = [homogen_name(x) for x in X_te1.columns.values.tolist()]
y_te1.columns = [homogen_name(x) for x in y_te1.columns.values.tolist()]


# Finally, we sort the data according to sensor's longitude
X_tr1_1 = X_tr1[longs.index]
y_tr1_1 = y_tr1[longs.index]
X_te1_1 = X_te1[longs.index]
y_te1_1 = y_te1[longs.index]

/home/davidgg/anaconda3/envs/solar/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Now we specify which sensor do we want to predict and test.

(In the future, we need to discuss how are we going to predict, if just by looping over each sensor, or just give a vectorial prediction)

In [2]:

# from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential, Model
# from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape, Add, Multiply, Subtract, Dropout
# from keras.layers import Conv2D, MaxPooling2D, LocallyConnected1D, Conv1D, UpSampling1D, MaxPooling1D, Dot, Concatenate

# from keras import backend as K

Model architecture is defined below.

Some highlights:
* Locally connected works better than pure convolutional at the first layers (probably because the sensors at not located in a uniform grid)
* Trick to improve acc: add a final layer combining the convolutional prediction with the persistance prediction, so in case the input is "strange", the model could learn to output the persistance prediction (i.e., the previous time-step), which is somewhat reasonable

In [17]:
def make_model_sensor(index_sensor, n_sensors=17):
    ''' Returns a model using all the sensors to predict index_sensor '''
    xin = Input(shape=(n_sensors,1), name='main_input')
    x = LocallyConnected1D(8, 7, data_format = 'channels_last', padding='valid')(xin)
    x = Activation('relu')(x)
    x = LocallyConnected1D(16, 5, data_format = 'channels_last', padding='valid')(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 3, data_format = 'channels_last', padding='causal')(x)
    xl = Flatten()(x)
    xl = Dropout(0.2)(xl)
    xo = Dense(1)(xl)

    # use date info here?
    xinf = Flatten()(xin)
    s  = Dense(5)(xinf)
    s = Activation('tanh')(s)
    s = Dense(2)(s)
    s = Activation('softmax')(s)

    # sort of residual connection
    xin_0 = Activation('relu')(xin)
    xin_1 = Lambda(lambda x : x[:,index_sensor,:])(xin_0)
    xo_m = Dot(axes=1)([Concatenate()([xo,xin_1]), s])
    xo_m = Activation('relu')(xo_m)

    model = Model(inputs=[xin], outputs=[xo_m])
    return model

In [18]:
lr = 0.0001
lr = 0.0001
opt = keras.optimizers.Adam(lr=lr)

# We add a callback to log metrics and another one to schedule the learning rate

#see clr.py in this same folder
from clr import CyclicLR

c1 = keras.callbacks.BaseLogger(stateful_metrics=None)
c2 = CyclicLR(step_size=250, base_lr=lr)
c3 = keras.callbacks.History()

Now we are ready to train. The below configuration should take 2 minutes in a 16 core CPU
(no GPU needed). We are using a huge batch-size to speed up things

In [19]:
n_sensors = 17

def to_array(sensor='AP5', val=0.1):
    ''' Converts dataframe to numpy array for predicting any given sensor. val specifies the fraction
    of training samples to be used as validation. '''
    X_tr1_1_np = X_tr1_1.values
    y_tr1_1_np = y_tr1_1[sensor].values
    
    #val_idx = int((1 - val)*len(y_tr1_1_np))

    X_te1_1_np = X_te1_1.values
    y_te1_1_np = y_te1_1[sensor].values
    
    #return X_tr1_1_np[:val_idx], y_tr1_1_np[:val_idx], X_tr1_1_np[val_idx:], y_tr1_1_np[val_idx:], X_te1_1_np, y_te1_1_np
    return X_tr1_1_np, y_tr1_1_np, X_te1_1_np, y_te1_1_np

In [20]:
batch_size = 1 << 11   # as big as possible so we can explore many models
epochs = 1 << 5

In [22]:
from sklearn.model_selection import TimeSeriesSplit

longs_np = longs.index.values

In [23]:
def train_and_test_sensor(id_sensor=4):
    X_tr, y_tr, X_te, y_te = to_array(sensor=longs_np[id_sensor])
    
    
    # Validation using TS split (just to obtain different MAE estimations, no hyperoptimization for the moment)
    for tr_idx, va_idx in TimeSeriesSplit(n_splits=5).split(X_tr):
        model = make_model_sensor(id_sensor, n_sensors=17)
        model.compile(opt, loss='mean_absolute_error')
        model.fit(np.atleast_3d(X_tr[tr_idx]), y_tr[tr_idx], batch_size=batch_size, epochs=epochs, validation_data=
              (np.atleast_3d(X_tr[va_idx]),y_tr[va_idx]), callbacks=[c2, c3], verbose=0)
        print('MAE_val ', c3.history['val_loss'][-1])
    
    # Testing
    model = make_model_sensor(id_sensor, n_sensors=17)
    model.compile(opt, loss='mean_absolute_error')
    model.fit(np.atleast_3d(X_tr), y_tr, batch_size=batch_size, epochs=epochs, validation_data=
              (np.atleast_3d(X_te),y_te), callbacks=[c2, c3], verbose=0)
    
    print('MAE_test ', c3.history['val_loss'][-1])
    return longs_np[id_sensor], c3.history['val_loss'][-1]

In [24]:
maes = {}
for i in range(len(longs_np)):
    print(i, longs_np[i])
    sensor, mae = train_and_test_sensor(i)
    maes[sensor] = mae 

0 AP7
MAE_val  75.80979736120578
MAE_val  51.144046569770296
MAE_val  23.682809347927705
MAE_val  48.48229516641444
MAE_val  77.36800727015067
MAE_test  56.7753012248093
1 AP6
MAE_val  74.61921432023048
MAE_val  49.396017061638695
MAE_val  22.354893112182616
MAE_val  45.92701640405692
MAE_val  76.46450659658674
MAE_test  56.88833060529433
2 AP4
MAE_val  66.64517080325065
MAE_val  44.6123261167234
MAE_val  22.224348633040396
MAE_val  46.57306654620193
MAE_val  70.25389557251026
MAE_test  53.94465374119383
3 AP3
MAE_val  53.7360220670414
MAE_val  40.76224770307509
MAE_val  21.362736480505344
MAE_val  73.05565114166914
MAE_val  46.59384850912657
MAE_test  6.656649084376051
4 AP5
MAE_val  55.24970045057928
MAE_val  36.355663239521924
MAE_val  22.087352481755055
MAE_val  37.961037127287135
MAE_val  57.43544904833747
MAE_test  41.17154105209058
5 AP1
MAE_val  53.48813401489421
MAE_val  41.610838796882916
MAE_val  20.758525616729365
MAE_val  41.51859642320799
MAE_val  60.26669131750933
MAE_te

In [25]:
maes = pd.Series(maes, name='MAE').sort_values()

In [26]:
maes

AP3      6.656649
DH8     40.139457
DH11    40.680962
AP5     41.171541
DH10    43.261680
DH6     43.378301
DH9     44.516583
DH7     45.890828
DH4     48.433724
DH5     49.663516
DH1     50.189021
DH3     50.249993
DH2     51.129463
AP1     51.893635
AP4     53.944654
AP7     56.775301
AP6     56.888331
Name: MAE, dtype: float64

['t-9', 't-8', 't-7', 't-6', 't-5', 't-4', 't-3', 't-2', 't-1']

## Convolutional 1D over longitude (time-slices are channels). 

This is a generalization of Victor's model. Taking only one time-slice, the last one, is Victor's model.

MAE

* AP7 : 55.17 (+/- 19.88) / List: [  75.48,  51.18,  23.41,  48.40,  77.37 ]
* AP6 : 54.53 (+/- 21.04) / List: [  75.35,  49.96,  22.55,  44.91,  79.89 ]
* AP4 : 50.27 (+/- 17.21) / List: [  67.10,  44.72,  22.43,  47.26,  69.85 ]
* AP3 : 49.31 (+/- 14.34) / List: [  58.20,  50.72,  21.89,  62.78,  52.95 ]
* AP5 : 47.06 (+/- 14.29) / List: [  61.25,  51.57,  22.26,  41.00,  59.22 ]
* AP1 : 47.96 (+/- 15.01) / List: [  62.10,  44.65,  21.55,  48.79,  62.72 ]
* DH5 : 50.61 (+/- 17.53) / List: [  74.30,  46.24,  22.58,  46.82,  63.09 ]
* DH3 : 52.80 (+/- 18.95) / List: [  57.66,  44.22,  21.79,  61.98,  78.32 ]
* DH4 : 49.47 (+/- 15.73) / List: [  61.41,  42.79,  25.75,  46.12,  71.27 ]
* DH11: 41.83 (+/- 12.06) / List: [  53.32,  38.17,  20.84,  43.15,  53.65 ]
* DH2 : 50.61 (+/- 16.71) / List: [  68.28,  49.07,  22.53,  46.06,  67.10 ]
* DH10: 47.52 (+/- 15.53) / List: [  53.72,  37.31,  22.82,  57.39,  66.39 ]
* DH8 : 42.17 (+/- 12.79) / List: [  52.66,  39.26,  21.01,  40.03,  57.88 ]
* DH6 : 46.11 (+/- 13.56) / List: [  58.33,  41.01,  23.45,  46.48,  61.27 ]
* DH7 : 49.78 (+/- 16.07) / List: [  66.84,  44.01,  22.92,  49.66,  65.44 ]
* DH9 : 46.36 (+/- 14.50) / List: [  58.73,  40.84,  21.81,  47.83,  62.58 ]
* DH1 : 50.41 (+/- 16.77) / List: [  67.71,  48.95,  21.88,  46.50,  67.00 ]

In [19]:
def to_array_cov1d_long(X,y,chosen_times, sensor='AP5', sort_time = True):
    ''' Converts dataframes to numpy array for predictions on a sensor. 
    If sort_time = True, chosen_times is sorted in ascending order. The last input is the lattest time slice.
    Output shape X_np: (items, time_slice, sensor) 'channels_last'
    Output shape y_np : (items,)
    '''
    if sort_time:
        chosen_times = sorted(chosen_times,reverse=True)
    X_np = []
    for time in chosen_times:
        X_np.append(np.expand_dims(X[time].to_numpy(),axis=2))
    X_np = np.concatenate(X_np,axis=2)
    y_np = y[sensor].to_numpy()

    assert X_np.shape[2] == len(chosen_times),'X_np does not have the correct shape'
    assert X_np.shape[0] == y_np.shape[0],'X and y do not have the same dimensions'
    return X_np, y_np

In [20]:
def make_model_sensor_conv1d_long(index_sensor, n_sensors=17, time_slices = 1):
    ''' Returns a model using all the sensors to predict index_sensor '''
    xin = Input(shape=(n_sensors,time_slices), name='main_input')
    '''
    When using this layer as the first layer in a model, provide an input_shape argument 
    (tuple of integers or None, does not include the batch axis), e.g. input_shape=(10, 128) 
    for time series sequences of 10 time steps with 128 features per step in data_format="channels_last", 
    or (None, 128) for variable-length sequences with 128 features per step.
    '''
    # The convolution is across time, and each sensor is a channel
    x = LocallyConnected1D(20, 3, data_format = 'channels_last', padding='valid')(xin)
    x = Activation('relu')(x)
    x = LocallyConnected1D(12, 3, data_format = 'channels_last', padding='valid')(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 3, data_format = 'channels_last', padding='causal')(x)
    xl = Flatten()(x)
    xl = Dropout(0.2)(xl)
    xo = Dense(1)(xl)

    # use date info here?
    xinf = Flatten()(xin)
    s  = Dense(5)(xinf)
    s = Activation('tanh')(s)
    s = Dense(2)(s)
    s = Activation('softmax')(s)

    # sort of residual connection, we only take the last timestamp of the given sensor x[:,-1:,index_sensor]
    xin_0 = Activation('relu')(xin)
    xin_1 = Lambda(lambda x : x[:,index_sensor,-1:])(xin_0)
#     xin_1 = Flatten()(xin_1)
    xo_m = Dot(axes=1)([Concatenate()([xo,xin_1]), s])
    xo_m = Activation('relu')(xo_m)

    model = Model(inputs=[xin], outputs=[xo_m])
    return model

Check

In [21]:
model = make_model_sensor_conv1d_long(0, n_sensors=n_sensors,time_slices=len(chosen_times))
# model.summary()
trainable_count = int(
    np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
non_trainable_count = int(
    np.sum([K.count_params(p) for p in set(model.non_trainable_weights)]))

print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

Total params: 20,299
Trainable params: 20,299
Non-trainable params: 0


In [ ]:
maes = {}
mae_val_list = np.zeros((n_sensors,5))
for i in range(len(longs_np)):
    print(i, longs_np[i])
    sensor, mae_list = train_and_test_sensor(X_train,y_train,i, chosen_times, 
                                             to_array=to_array_cov1d_long,
                                             make_model_sensor=make_model_sensor_conv1d_long)
    maes[sensor] = mae_list
    mae_val_list[i]=mae_list
mae_val_list = mae_val_list.T

0 AP7


In [ ]:
print_mae_sensors(mae_val_list,longs_np)

## Convolutional 1D over time (longitudes are channels)

MAE

* AP7 : 55.37 (+/- 19.78) / List: [  75.58,  51.40,  23.79,  48.58,  77.51 ]
* AP6 : 54.45 (+/- 20.79) / List: [  76.13,  49.88,  22.20,  45.95,  78.06 ]
* AP4 : 50.82 (+/- 18.55) / List: [  68.85,  45.95,  21.69,  44.74,  72.86 ]
* AP3 : 36.95 (+/- 21.29) / List: [  61.12,  50.96,  23.21,  47.08,   2.40 ]
* AP5 : 44.82 (+/- 13.32) / List: [  59.34,  40.75,  24.09,  40.56,  59.38 ]
* AP1 : 49.84 (+/- 17.73) / List: [  58.54,  48.88,  20.17,  47.13,  74.49 ]
* DH5 : 46.59 (+/- 12.83) / List: [  58.18,  45.05,  23.37,  47.67,  58.70 ]
* DH3 : 49.64 (+/- 17.88) / List: [  57.56,  43.41,  20.51,  51.70,  75.01 ]
* DH4 : 50.32 (+/- 16.93) / List: [  59.15,  44.20,  23.91,  49.11,  75.20 ]
* DH11: 45.87 (+/- 17.79) / List: [  52.50,  37.60,  19.84,  45.41,  74.01 ]
* DH2 : 51.87 (+/- 17.44) / List: [  69.01,  51.73,  23.02,  45.03,  70.54 ]
* DH10: 48.72 (+/- 18.59) / List: [  56.34,  38.21,  20.58,  52.30,  76.19 ]
* DH8 : 42.33 (+/- 13.07) / List: [  53.92,  40.09,  19.18,  42.74,  55.72 ]
* DH6 : 48.83 (+/- 18.52) / List: [  75.91,  39.04,  21.54,  47.11,  60.57 ]
* DH7 : 48.42 (+/- 14.74) / List: [  61.84,  42.99,  23.54,  49.27,  64.46 ]
* DH9 : 46.22 (+/- 15.33) / List: [  59.11,  39.12,  21.51,  46.48,  64.88 ]
* DH1 : 51.45 (+/- 17.41) / List: [  73.68,  49.40,  23.50,  45.20,  65.49 ]

In [131]:
def to_array_cov1d_time(X,y,chosen_times, sensor='AP5', sort_time = True):
    ''' Converts dataframes to numpy array for predictions on a sensor. 

    Output shape X_np: (items, time_slice, sensor) 'channels_last'
    Output shape y_np : (items,)
    '''
    X_np = []
    for time in chosen_times:
        X_np.append(np.expand_dims(X[time].to_numpy(),axis=2))
    X_np = np.swapaxes(np.concatenate(X_np,axis=2),1,2)
    y_np = y[sensor].to_numpy()
    if sort_time:
        X_np = X_np[:,::-1]  
    assert X_np.shape[1] == len(chosen_times),'X_np does not have the correct shape'
    assert X_np.shape[0] == y_np.shape[0],'X and y do not have the same dimensions'
    return X_np, y_np

In [132]:
def make_model_sensor_conv1d_time(index_sensor, n_sensors=17, time_slices = 9):
    ''' Returns a model using all the sensors to predict index_sensor '''
    xin = Input(shape=(time_slices,n_sensors), name='main_input')
    '''
    When using this layer as the first layer in a model, provide an input_shape argument 
    (tuple of integers or None, does not include the batch axis), e.g. input_shape=(10, 128) 
    for time series sequences of 10 time steps with 128 features per step in data_format="channels_last", 
    or (None, 128) for variable-length sequences with 128 features per step.
    '''
    # The convolution is across time, and each sensor is a channel
    x = LocallyConnected1D(20, 3, data_format = 'channels_last', padding='valid')(xin)
    x = Activation('relu')(x)
    x = LocallyConnected1D(12, 3, data_format = 'channels_last', padding='valid')(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 3, data_format = 'channels_last', padding='causal')(x)
    xl = Flatten()(x)
    xl = Dropout(0.2)(xl)
    xo = Dense(1)(xl)

    # use date info here?
    xinf = Flatten()(xin)
    s  = Dense(5)(xinf)
    s = Activation('tanh')(s)
    s = Dense(2)(s)
    s = Activation('softmax')(s)

    # sort of residual connection, we only take the last timestamp of the given sensor x[:,-1:,index_sensor]
    xin_0 = Activation('relu')(xin)
    xin_1 = Lambda(lambda x : x[:,-1:,index_sensor])(xin_0)
#     xin_1 = Flatten()(xin_1)
    xo_m = Dot(axes=1)([Concatenate()([xo,xin_1]), s])
    xo_m = Activation('relu')(xo_m)

    model = Model(inputs=[xin], outputs=[xo_m])
    return model

Check

In [133]:
model = make_model_sensor_conv1d_time(0, n_sensors=n_sensors,time_slices=len(chosen_times))
# model.summary()
trainable_count = int(
    np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
non_trainable_count = int(
    np.sum([K.count_params(p) for p in set(model.non_trainable_weights)]))

print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

Total params: 13,067
Trainable params: 13,067
Non-trainable params: 0


In [134]:
maes = {}
for i in range(len(longs_np)):
    print(i, longs_np[i])
    sensor, mae_list = train_and_test_sensor(X_train,y_train,i, to_array=to_array_cov1d_time,make_model_sensor=make_model_sensor_conv1d_time)
    maes[sensor] = mae_list
    
print('MAE\n=========')
for sensor,mae_list in maes.items():
    print("{:4}: {:0.2f} (+/- {:0.2f})".format(sensor, mae_list.mean(), mae_list.std()),'/ List: [',', '.join(['{:6.2f}'.format(mae) for mae in mae_list]),']') 

0 AP7


/home/davidgg/anaconda3/envs/solar/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.767133). Check your callbacks.
  % delta_t_median)


1 AP6
2 AP4
3 AP3
4 AP5
5 AP1
6 DH5
7 DH3
8 DH4
9 DH11
10 DH2
11 DH10
12 DH8
13 DH6
14 DH7
15 DH9
16 DH1
MAE
AP7 : 55.37 (+/- 19.78) / List: [  75.58,  51.40,  23.79,  48.58,  77.51 ]
AP6 : 54.45 (+/- 20.79) / List: [  76.13,  49.88,  22.20,  45.95,  78.06 ]
AP4 : 50.82 (+/- 18.55) / List: [  68.85,  45.95,  21.69,  44.74,  72.86 ]
AP3 : 36.95 (+/- 21.29) / List: [  61.12,  50.96,  23.21,  47.08,   2.40 ]
AP5 : 44.82 (+/- 13.32) / List: [  59.34,  40.75,  24.09,  40.56,  59.38 ]
AP1 : 49.84 (+/- 17.73) / List: [  58.54,  48.88,  20.17,  47.13,  74.49 ]
DH5 : 46.59 (+/- 12.83) / List: [  58.18,  45.05,  23.37,  47.67,  58.70 ]
DH3 : 49.64 (+/- 17.88) / List: [  57.56,  43.41,  20.51,  51.70,  75.01 ]
DH4 : 50.32 (+/- 16.93) / List: [  59.15,  44.20,  23.91,  49.11,  75.20 ]
DH11: 45.87 (+/- 17.79) / List: [  52.50,  37.60,  19.84,  45.41,  74.01 ]
DH2 : 51.87 (+/- 17.44) / List: [  69.01,  51.73,  23.02,  45.03,  70.54 ]
DH10: 48.72 (+/- 18.59) / List: [  56.34,  38.21,  20.58,  52.30, 

## Convolutional 2D

MAE

* AP7 : 55.01 (+/- 19.98) / List: [  75.60,  50.97,  23.25,  47.91,  77.31 ]
* AP6 : 53.39 (+/- 20.02) / List: [  74.55,  49.38,  22.12,  45.27,  75.66 ]
* AP4 : 51.76 (+/- 19.38) / List: [  75.41,  45.62,  23.04,  43.35,  71.37 ]
* AP3 : 47.05 (+/- 15.50) / List: [  55.63,  41.67,  22.32,  69.11,  46.51 ]
* AP5 : 43.60 (+/- 13.57) / List: [  57.45,  37.10,  23.18,  40.52,  59.74 ]
* AP1 : 49.19 (+/- 16.05) / List: [  61.87,  43.01,  21.17,  53.99,  65.94 ]
* DH5 : 49.11 (+/- 14.01) / List: [  59.44,  50.70,  22.87,  49.91,  62.65 ]
* DH3 : 50.91 (+/- 18.47) / List: [  60.31,  42.43,  21.97,  52.38,  77.47 ]
* DH4 : 48.52 (+/- 15.05) / List: [  58.14,  44.60,  23.06,  48.88,  67.93 ]
* DH11: 42.94 (+/- 13.00) / List: [  52.68,  38.06,  20.60,  45.62,  57.76 ]
* DH2 : 50.06 (+/- 16.43) / List: [  66.78,  48.65,  22.71,  44.97,  67.20 ]
* DH10: 48.21 (+/- 17.12) / List: [  55.41,  41.07,  20.67,  51.30,  72.61 ]
* DH8 : 44.64 (+/- 15.81) / List: [  64.09,  38.52,  20.98,  39.28,  60.31 ]
* DH6 : 44.13 (+/- 13.67) / List: [  55.36,  39.30,  21.63,  43.57,  60.80 ]
* DH7 : 48.47 (+/- 16.09) / List: [  65.76,  45.60,  22.05,  44.02,  64.91 ]
* DH9 : 46.47 (+/- 15.96) / List: [  59.62,  40.53,  21.27,  43.88,  67.03 ]
* DH1 : 50.00 (+/- 16.38) / List: [  66.84,  48.67,  22.85,  44.61,  67.03 ]


Preprocessing

In [99]:
def to_array_cov2d(X,y,chosen_times, sensor='AP5', sort_time = True):
    ''' Converts dataframe to numpy array for predicting any given sensor.
    The X array has several time slices, given in chosen_times and
    sorts them in ascending order.
    Output shape X_np : (items, time_slice, sensor, 1), 'channels_last'
    Output shape y_np : (items)
    '''
    X_np = []
    for time in chosen_times:
        X_np.append(np.expand_dims(X[time].to_numpy(),axis=2))
    X_np = np.swapaxes(np.concatenate(X_np,axis=2),1,2)
    y_np = y[sensor].to_numpy()
    if sort_time:
        X_np = X_np[:,::-1]  
    X_np = np.expand_dims(X_np,axis=3)
    assert X_np.shape[1] == len(chosen_times),'X_np does not have the correct shape'
    assert X_np.shape[0] == y_np.shape[0],'X and y do not have the same dimensions'
    return X_np, y_np

Define model architecture

In [100]:
def make_model_sensor_conv2d(index_sensor, n_sensors=17, time_slices = 2):
    '''
    Returns a model using all the sensors to predict index_sensor 
    Idea: We generate two features, one is a complex one, through convolutions xo.
    The other one is just the measurement of the same sensor at the last time xin_1.
    Then, we ponderate them with another feature of weights s.
    '''
    
    xin = Input(shape=(time_slices,n_sensors,1), name='main_input')
    # The convolution is across time, and each sensor is a channel
    '''
    When using this layer as the first layer in a model, provide an input_shape argument 
    (tuple of integers or None, does not include the batch axis), e.g. input_shape=(10, 128) 
    for time series sequences of 10 time steps with 128 features per step in data_format="channels_last", 
    or (None, 128) for variable-length sequences with 128 features per step.
    '''
    x = LocallyConnected2D(10, 3, data_format = 'channels_last', padding='valid')(xin)
    x = Activation('relu')(x)
    x = LocallyConnected2D(5, 3, data_format = 'channels_last', padding='valid')(x)
    x = Activation('relu')(x)
    x = Conv2D(12, 3, data_format = 'channels_last', padding='valid')(x)
    xl = Flatten()(x)
    xl = Dropout(0.2)(xl)
    xo = Dense(1)(xl)
    # sort of residual connection, we only take the last timestamp of the given sensor x[:,index_sensor,-1:]
    xin_0 = Activation('relu')(xin)
    xin_1 = Lambda(lambda x : x[:,-1:,index_sensor,-1])(xin_0)
    # Weights of the weighted mean
    # use date info here?
    xinf = Flatten()(xin)
    s  = Dense(5)(xinf)
    s = Activation('tanh')(s)
    s = Dense(2)(s)
    s = Activation('softmax')(s)
    # Weighted mean
    xo_m = Dot(axes=1)([Concatenate()([xo,xin_1]), s])
    # The ouput must be a positive quantity
    xo_m = Activation('relu')(xo_m)

    model = Model(inputs=[xin], outputs=[xo_m])
    return model

In [101]:
model = make_model_sensor_conv2d(0, n_sensors=n_sensors,time_slices=len(chosen_times))
# model.summary()
trainable_count = int(
    np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
non_trainable_count = int(
    np.sum([K.count_params(p) for p in set(model.non_trainable_weights)]))

print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

Total params: 41,806
Trainable params: 41,806
Non-trainable params: 0


0 AP7

MAE_val  75.80979736120578

MAE_val  51.144046569770296

MAE_val  23.682809347927705

MAE_val  48.48229516641444

MAE_val  77.36800727015067

MAE_test  56.7753012248093

In [135]:
maes = {}
for i in range(len(longs_np)):
    print(i, longs_np[i])
    sensor, mae_list = train_and_test_sensor(X_train,y_train,i,to_array = to_array_cov2d,make_model_sensor=make_model_sensor_conv2d,verbose=1)
    maes[sensor] = mae_list
    
print('MAE\n=========')
for sensor,mae_list in maes.items():
    print("{:4}: {:0.2f} (+/- {:0.2f})".format(sensor, mae_list.mean(), mae_list.std()),'/ List: [',', '.join(['{:6.2f}'.format(mae) for mae in mae_list]),']') 

0 AP7
Train on 74985 samples, validate on 74980 samples
Epoch 1/32
74985/74985 [==============================] - 8s 109us/step - loss: 88.2718 - val_loss: 76.2590
Epoch 2/32
74985/74985 [==============================] - 3s 34us/step - loss: 77.9687 - val_loss: 76.1576
Epoch 3/32
74985/74985 [==============================] - 3s 37us/step - loss: 77.7874 - val_loss: 75.9325
Epoch 4/32
74985/74985 [==============================] - 3s 36us/step - loss: 77.5432 - val_loss: 75.8022
Epoch 5/32
74985/74985 [==============================] - 3s 36us/step - loss: 77.4060 - val_loss: 75.6987
Epoch 6/32
74985/74985 [==============================] - 3s 36us/step - loss: 77.2458 - val_loss: 75.7396
Epoch 7/32
74985/74985 [==============================] - 3s 36us/step - loss: 77.3159 - val_loss: 75.6297
Epoch 8/32
74985/74985 [==============================] - 3s 36us/step - loss: 77.1420 - val_loss: 75.6467
Epoch 9/32
74985/74985 [==============================] - 3s 36us/step - loss: 77.0889 

224945/224945 [==============================] - 6s 28us/step - loss: 67.4238 - val_loss: 23.3071
Epoch 25/32
224945/224945 [==============================] - 6s 29us/step - loss: 67.4081 - val_loss: 23.4330
Epoch 26/32
224945/224945 [==============================] - 6s 29us/step - loss: 67.4553 - val_loss: 23.2565
Epoch 27/32
224945/224945 [==============================] - 6s 29us/step - loss: 67.5769 - val_loss: 23.3057
Epoch 28/32
224945/224945 [==============================] - 6s 29us/step - loss: 67.4069 - val_loss: 23.2547
Epoch 29/32
224945/224945 [==============================] - 6s 29us/step - loss: 67.3455 - val_loss: 23.2102
Epoch 30/32
224945/224945 [==============================] - 7s 29us/step - loss: 67.3556 - val_loss: 23.2481
Epoch 31/32
224945/224945 [==============================] - 6s 29us/step - loss: 67.4120 - val_loss: 23.3228
Epoch 32/32
224945/224945 [==============================] - 6s 29us/step - loss: 67.4105 - val_loss: 23.2491
MAE_val  23.2491337665

74985/74985 [==============================] - 3s 35us/step - loss: 76.0705 - val_loss: 75.3312
Epoch 10/32
74985/74985 [==============================] - 3s 35us/step - loss: 75.9999 - val_loss: 75.2275
Epoch 11/32
74985/74985 [==============================] - 3s 36us/step - loss: 76.0119 - val_loss: 75.4856
Epoch 12/32
74985/74985 [==============================] - 3s 35us/step - loss: 76.1097 - val_loss: 75.1650
Epoch 13/32
74985/74985 [==============================] - 3s 34us/step - loss: 75.9670 - val_loss: 75.3031
Epoch 14/32
74985/74985 [==============================] - 3s 36us/step - loss: 76.0686 - val_loss: 76.0606
Epoch 15/32
74985/74985 [==============================] - 3s 34us/step - loss: 76.0617 - val_loss: 75.0933
Epoch 16/32
74985/74985 [==============================] - 3s 37us/step - loss: 76.2445 - val_loss: 75.3035
Epoch 17/32
74985/74985 [==============================] - 3s 35us/step - loss: 76.0611 - val_loss: 74.9801
Epoch 18/32
74985/74985 [===============

224945/224945 [==============================] - 7s 30us/step - loss: 66.6675 - val_loss: 22.3947
Epoch 19/32
224945/224945 [==============================] - 7s 30us/step - loss: 66.8013 - val_loss: 22.2834
Epoch 20/32
224945/224945 [==============================] - 7s 29us/step - loss: 66.6949 - val_loss: 22.1750
Epoch 21/32
224945/224945 [==============================] - 6s 28us/step - loss: 66.5713 - val_loss: 22.1840
Epoch 22/32
224945/224945 [==============================] - 7s 29us/step - loss: 66.6011 - val_loss: 22.1523
Epoch 23/32
224945/224945 [==============================] - 6s 29us/step - loss: 66.5630 - val_loss: 22.2053
Epoch 24/32
224945/224945 [==============================] - 7s 29us/step - loss: 66.6327 - val_loss: 22.1962
Epoch 25/32
224945/224945 [==============================] - 6s 29us/step - loss: 66.4782 - val_loss: 22.1019
Epoch 26/32
224945/224945 [==============================] - 6s 29us/step - loss: 66.3975 - val_loss: 22.0802
Epoch 27/32
224945/224

374905/374905 [==============================] - 10s 28us/step - loss: 52.1863 - val_loss: 75.8710
Epoch 28/32
374905/374905 [==============================] - 10s 27us/step - loss: 51.9131 - val_loss: 75.7265
Epoch 29/32
374905/374905 [==============================] - 11s 28us/step - loss: 52.0626 - val_loss: 76.5461
Epoch 30/32
374905/374905 [==============================] - 11s 28us/step - loss: 52.0576 - val_loss: 75.6866
Epoch 31/32
374905/374905 [==============================] - 10s 27us/step - loss: 51.7994 - val_loss: 75.6644
Epoch 32/32
374905/374905 [==============================] - 10s 27us/step - loss: 52.0157 - val_loss: 75.6597
MAE_val  75.6596981110653
2 AP4
Train on 74985 samples, validate on 74980 samples
Epoch 1/32
74985/74985 [==============================] - 8s 112us/step - loss: 87.8418 - val_loss: 76.1702
Epoch 2/32
74985/74985 [==============================] - 3s 37us/step - loss: 76.3289 - val_loss: 76.1754
Epoch 3/32
74985/74985 [=========================

...

374905/374905 [==============================] - 12s 32us/step - loss: 43.6120 - val_loss: 63.3397
Epoch 6/32
374905/374905 [==============================] - 12s 31us/step - loss: 42.7295 - val_loss: 63.5983
Epoch 7/32
374905/374905 [==============================] - 12s 32us/step - loss: 42.9147 - val_loss: 62.1793
Epoch 8/32
374905/374905 [==============================] - 12s 32us/step - loss: 42.2288 - val_loss: 60.7689
Epoch 9/32
374905/374905 [==============================] - 12s 31us/step - loss: 41.8515 - val_loss: 63.5289
Epoch 10/32
374905/374905 [==============================] - 12s 31us/step - loss: 42.4335 - val_loss: 62.6004
Epoch 11/32
374905/374905 [==============================] - 12s 31us/step - loss: 41.5677 - val_loss: 61.0498
Epoch 12/32
374905/374905 [==============================] - 11s 30us/step - loss: 41.4566 - val_loss: 61.6984
Epoch 13/32
374905/374905 [==============================] - 12s 31us/step - loss: 41.5757 - val_loss: 60.8021
MAE_val  60.80214

Epoch 8/32
224945/224945 [==============================] - 7s 33us/step - loss: 59.8664 - val_loss: 21.8906
Epoch 9/32
224945/224945 [==============================] - 7s 32us/step - loss: 59.8530 - val_loss: 25.1034
Epoch 10/32
224945/224945 [==============================] - 7s 33us/step - loss: 59.9859 - val_loss: 23.2614
Epoch 11/32
224945/224945 [==============================] - 8s 34us/step - loss: 59.7939 - val_loss: 22.3997
Epoch 12/32
224945/224945 [==============================] - 7s 33us/step - loss: 59.7764 - val_loss: 21.8271
Epoch 13/32
224945/224945 [==============================] - 8s 33us/step - loss: 59.0350 - val_loss: 22.7070
Epoch 14/32
224945/224945 [==============================] - 7s 33us/step - loss: 59.2235 - val_loss: 21.6309
Epoch 15/32
224945/224945 [==============================] - 8s 33us/step - loss: 59.0592 - val_loss: 21.9943
Epoch 16/32
224945/224945 [==============================] - 7s 33us/step - loss: 58.7676 - val_loss: 21.6492
Epoch 17/32


Epoch 2/32
149965/149965 [==============================] - 5s 37us/step - loss: 70.9397 - val_loss: 47.4874
Epoch 3/32
149965/149965 [==============================] - 5s 36us/step - loss: 64.5874 - val_loss: 43.1570
Epoch 4/32
149965/149965 [==============================] - 6s 37us/step - loss: 62.1455 - val_loss: 43.0514
Epoch 5/32
149965/149965 [==============================] - 5s 35us/step - loss: 61.6667 - val_loss: 42.3339
Epoch 6/32
149965/149965 [==============================] - 5s 36us/step - loss: 62.1900 - val_loss: 43.5206
Epoch 7/32
149965/149965 [==============================] - 5s 35us/step - loss: 61.3036 - val_loss: 45.1385
Epoch 8/32
149965/149965 [==============================] - 5s 37us/step - loss: 61.0085 - val_loss: 41.2832
Epoch 9/32
149965/149965 [==============================] - 5s 36us/step - loss: 59.9090 - val_loss: 41.4316
Epoch 10/32
149965/149965 [==============================] - 6s 37us/step - loss: 59.4529 - val_loss: 42.0951
Epoch 11/32
149965

Epoch 6/32
74985/74985 [==============================] - 3s 42us/step - loss: 71.0487 - val_loss: 69.7105
Epoch 7/32
74985/74985 [==============================] - 3s 42us/step - loss: 70.3921 - val_loss: 69.8012
Epoch 8/32
74985/74985 [==============================] - 3s 42us/step - loss: 69.6572 - val_loss: 69.0530
Epoch 9/32
74985/74985 [==============================] - 3s 42us/step - loss: 68.9812 - val_loss: 68.7063
Epoch 10/32
74985/74985 [==============================] - 3s 43us/step - loss: 68.6252 - val_loss: 68.2839
Epoch 11/32
74985/74985 [==============================] - 3s 42us/step - loss: 68.4681 - val_loss: 68.4191
Epoch 12/32
74985/74985 [==============================] - 3s 44us/step - loss: 68.1738 - val_loss: 67.9698
Epoch 13/32
74985/74985 [==============================] - 3s 43us/step - loss: 68.0850 - val_loss: 67.8857
Epoch 14/32
74985/74985 [==============================] - 3s 41us/step - loss: 68.0376 - val_loss: 67.8100
Epoch 15/32
74985/74985 [=======

224945/224945 [==============================] - 7s 32us/step - loss: 60.7613 - val_loss: 21.6973
Epoch 27/32
224945/224945 [==============================] - 8s 33us/step - loss: 61.1386 - val_loss: 21.6122
Epoch 28/32
224945/224945 [==============================] - 7s 32us/step - loss: 60.8609 - val_loss: 21.8155
Epoch 29/32
224945/224945 [==============================] - 8s 33us/step - loss: 60.6400 - val_loss: 21.9449
Epoch 30/32
224945/224945 [==============================] - 8s 34us/step - loss: 60.6454 - val_loss: 22.6234
Epoch 31/32
224945/224945 [==============================] - 8s 34us/step - loss: 60.9643 - val_loss: 23.7224
Epoch 32/32
224945/224945 [==============================] - 7s 33us/step - loss: 60.8951 - val_loss: 22.8513
MAE_val  22.851292470920306
Train on 299925 samples, validate on 74980 samples
Epoch 1/32
299925/299925 [==============================] - 18s 61us/step - loss: 56.8498 - val_loss: 47.4374
Epoch 2/32
299925/299925 [===========================